<a href="https://colab.research.google.com/github/isnanramalia/cryptography2023/blob/main/RSA/RSA_Isna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Urutan Pemakaian
1. Upload Gambar kalian di google collab
2. Rubah Gambar ke base64. di kodingan atas
3. Kalian enkripsi hasil encoding tadi pakai nama file hasil encoding kalian
4. Kalian dekripsi dengan file hasil_enkripsi kalian
5. Check apakah gambar awal sudah sama dengan hasil dekripsi kalian
6. Cetak Gambar kalian

Saran aku coba coba dulu nilai p, q dan totientn nya kecil biar komputasi ringan

Untuk merubah image ke base64

# Image to Bytes

In [ ]:
from PIL import Image
import base64
from io import BytesIO

# Fungsi untuk membaca gambar dan mengonversinya ke format Base64
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        # Baca gambar sebagai bytes
        image_binary = image_file.read()

        # Encode gambar ke dalam format Base64
        image_base64 = base64.b64encode(image_binary).decode("utf-8")

    return image_base64

# Fungsi untuk mendekode gambar dari format Base64
def base64_to_image(base64_data, output_path):
    # Decode Base64 ke dalam bytes
    image_binary = base64.b64decode(base64_data.encode("utf-8"))

    # Baca bytes sebagai gambar menggunakan PIL
    image = Image.open(BytesIO(image_binary))

    # Simpan gambar ke dalam file
    image.save(output_path)

# Fungsi untuk memeriksa kesamaan gambar
def compare_images(original_image_path, base64_text):
    # Buka gambar asli
    original_image = Image.open(original_image_path)

    # Decode Base64 ke dalam bytes
    decoded_image_binary = base64.b64decode(base64_text.encode("utf-8"))

    # Baca bytes sebagai gambar menggunakan PIL
    decoded_image = Image.open(BytesIO(decoded_image_binary))

    # Periksa apakah ukuran gambar sama
    if original_image.size == decoded_image.size:
        print("Ukuran gambar sama.")
    else:
        print("Perhatian: Ukuran gambar tidak sama!")

    # Periksa apakah piksel-piksel gambar sama
    if original_image.tobytes() == decoded_image.tobytes():
        print("Gambar berhasil didecode dan sama dengan gambar asli.")
    else:
        print("Perhatian: Gambar hasil decoding tidak sama dengan gambar asli.")

# Fungsi untuk menyimpan hasil encoding Base64 ke dalam file teks
def save_base64_to_text(base64_data, output_path):
    with open(output_path, "w") as base64_file:
        base64_file.write(base64_data)

# Fungsi untuk mendekode teks Base64 dan membuat gambar
def base64_to_image_and_save(base64_data, output_path):
    # Decode Base64 ke dalam bytes
    image_binary = base64.b64decode(base64_data.encode("utf-8"))

    # Baca bytes sebagai gambar menggunakan PIL
    image = Image.open(BytesIO(image_binary))

    # Simpan gambar ke dalam file
    image.save(output_path)

# Menu
while True:
    print("\nMenu:")
    print("1. Merubah dari gambar ke teks base64")
    print("2. Mengecek kesamaan gambar")
    print("3. Membuat gambar dari teks base64")
    print("4. Keluar")

    choice = input("Pilih menu (1/2/3/4): ")

    if choice == "1":
        # Merubah dari gambar ke teks base64
        image_path = input("Masukkan path gambar: ")
        base64_data = image_to_base64(image_path)
        print("Hasil encoding Base64:")
        print("Sukses! Hasil encoding Base64 tersimpan")

        # Menyimpan hasil encoding Base64 ke dalam file teks (opsional)
        save_choice = input("Apakah Anda ingin menyimpan hasil encoding ke dalam file teks? (y/n): ")
        if save_choice.lower() == "y":
            text_file_path = input("Masukkan path untuk menyimpan hasil encoding (contoh: hasil_encoding.txt): ")
            save_base64_to_text(base64_data, text_file_path)

    elif choice == "2":
        # Mengecek kesamaan gambar
        original_image_path = input("Masukkan path gambar asli: ")
        base64_text = input("Masukkan path hasil dekripsi (contoh : hasil_dekripsi.txt): ")
        with open(base64_text, 'r') as file:
        # Baca seluruh konten file
            message = file.read()
        compare_images(original_image_path, message)

    elif choice == "3":
        # Membuat gambar dari teks Base64
        base64_text = input("Masukkan path hasil dekripsi (contoh : hasil_dekripsi.txt): ")
        with open(base64_text, 'r') as file:
        # Baca seluruh konten file
            message = file.read()
        output_image_path = input("Masukkan path untuk menyimpan gambar hasil decoding (contoh: hasil_gambar.jpg): ")
        base64_to_image_and_save(message, output_image_path)

    elif choice == "4":
        # Keluar dari program
        break

    else:
        print("Pilihan tidak valid. Silakan pilih 1, 2, 3, atau 4.")


# Kodingan RSA

In [1]:
import random
import math

# Fungsi untuk mengecek apakah suatu bilangan merupakan bilangan prima
def is_prime(number):
    if number < 2:
        return False
    for i in range(2, number // 2 + 1):
        if number % i == 0:
            return False
    return True

# Fungsi untuk mengecek apakah suatu bilangan merupakan bilangan prima
def get_prime_input(message):
    while True:
        try:
            user_input = int(input(message))
            if is_prime(user_input):
                return user_input
            else:
                print("Input is not a prime number. Try again.")
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

# Fungsi untuk mencari nilai d (private key) dari nilai e (public key) dan totient n
def mod_inverse(e, totient):
    for d in range(3, totient):
        if (d * e) % totient == 1:
            return d
    raise ValueError('No mod inverse for e: %d, totient: %d' % (e, totient))

def make_key():
    p = get_prime_input("Enter a prime number (p): ")
    q = get_prime_input("Enter a prime number (q): ")

    # p dan q tidak boleh sama
    while p == q:
        print("p and q cannot be the same number. Try again.")
        p = get_prime_input("Enter a prime number (p): ")
        q = get_prime_input("Enter a prime number (q): ")

    # nilai n adalah hasil perkalian p dan q dan boleh dibagikan
    n = p * q
    # nilai totient n adalah hasil perkalian p-1 dan q-1 tidak boleh dibagikan karena untuk mencari nilai d
    totient_n = (p - 1) * (q - 1)

    # nilai e adalah nilai yang tidak boleh dibagikan dan harus lebih besar dari 2 dan lebih kecil dari Pembagi persekutuan terbesar / GCD
    e = int(input(f"Enter a public exponent (e) such that 2 < e < {totient_n} and gcd(e, {totient_n}) = 1: "))

    while not (1 < e < totient_n and math.gcd(e, totient_n) == 1):
        print(f"Invalid value for e. Try again.")
        e = int(input(f"Enter a public exponent (e) such that 2 < e < {totient_n} and gcd(e, {totient_n}) = 1: "))

    d = mod_inverse(e, totient_n)

    print("Public key (e, n): ", e, n)
    print("Private key (d, n): ", d, n)


def encryption_menu():
    n = int(input("Masukan nilai n: "))
    e = int(input("Masukan kunci public e: "))
    input_path = input("Masukkan path file untuk dienkripsi (contoh: hasil_encoding.txt): ")
    process_path = input("Masukkan path untuk menyimpan process enkripsi (contoh: process_enkripsi.txt): ")
    output_path = input("Masukkan path untuk menyimpan hasil enkripsi (contoh: hasil_enkripsi.txt): ")

    with open(input_path, 'r') as file:
        # Baca seluruh konten file
        message = file.read()

    # Mengubah pesan menjadi nilai ASCII dan enkripsi
    ciphertext = []
    max_digit = 0
    values = []

    # Open the file in append mode
    with open(process_path, "a") as base64_file:
        # Iterate through each character in the message
        for i in range(0, len(message)):
            block = message[i]
            encoded_block = ord(block)  # Convert character to ASCII value
            encrypted_block = pow(encoded_block, e, n)

            # Append the value to the list
            values.append(encrypted_block)

            # Write the details of the current block to the file
            base64_file.write(f"Block '{block}' is {encoded_block}, {encoded_block}^{e} mod {n} is {encrypted_block}\n")

            current_digit = len(str(encrypted_block))
            max_digit = max(max_digit, current_digit)
            ciphertext.append(encrypted_block)

    ciphertext_str = ''.join([str(block).zfill(max_digit) for block in ciphertext])

     # Simpan hasil enkripsi ke dalam file teks
    with open(output_path, "w") as base64_file:
        base64_file.write(ciphertext_str)
        print("Sukses! Hasil enkripsi tersimpan")

    return max_digit


def decryption_menu(max_digit):
    n = int(input("Masukan nilai n: "))
    d = int(input("Masukan kunci private d: "))
    input_path = input("Masukkan path file untuk didekripsi (contoh: hasil_enkripsi.txt): ")
    process_path = input("Masukkan path untuk menyimpan process dekripsi (contoh process_dekripsi.txt): ")
    output_path = input("Masukkan path untuk menyimpan hasil dekripsi (contoh: hasil_dekripsi.txt): ")

    with open(input_path, 'r') as file:
        # Baca seluruh konten file
        isi_file = file.read()

    ciphertext_str = isi_file

    # Split ciphertext into blocks of size max_digit
    ciphertext_blocks = [int(ciphertext_str[i:i + max_digit]) for i in range(0, len(ciphertext_str), max_digit)]

    decrypted_message = ""

    values = []

    # Open the file in append mode
    with open(process_path, "a") as base64_file:
        # Iterate through each block in the ciphertext
        for block in ciphertext_blocks:
            decrypted_block = pow(block, d, n)
            decrypted_char = chr(decrypted_block)  # Convert ASCII value to character

            # Append the value to the list
            values.append(decrypted_block)

            # Write the details of the current block to the file
            base64_file.write(f"{block}^{d} mod {n} is {decrypted_block} which is '{decrypted_char}'\n")

            decrypted_message += decrypted_char

    # Simpan hasil dekripsi ke dalam file teks
    with open(output_path, "w") as base64_file:
        base64_file.write(decrypted_message)
        print("Sukses! Hasil dekripsi tersimpan")

# Fungsi untuk menghitung totient n dari dua bilangan prima p dan q
def calculate_totient(p, q):
    return (p - 1) * (q - 1)

# Fungsi untuk mencari bilangan prima yang relatif prima terhadap totient n
def generate_coprime(num_digits, totient_n):
    while True:
        random_prime = generate_random_prime(num_digits)
        if math.gcd(random_prime, totient_n) == 1:
            return random_prime


# Fungsi untuk mengenerate bilangan prima dengan jumlah digit yang diinginkan
def generate_random_prime(num_digits):
    min_value = 10 ** (num_digits - 1)
    max_value = (10 ** num_digits) - 1

    while True:
        random_prime = random.randint(min_value, max_value)
        if is_prime(random_prime):
            return random_prime

# Fungsi untuk menampilkan menu bilangan prima custom
def custom_prime_menu():
    num_digits = input("Masukkan jumlah digit bilangan prima yang diinginkan: ")
    # generate bilangan prima dengan jumlah digit yang diinginkan
    prime1 = generate_random_prime(int(num_digits))
    prime2 = generate_random_prime(int(num_digits))

    if prime1 is not None and prime2 is not None:
        print(f"Bilangan prima dengan {num_digits} digit (p): {prime1}")
        print(f"Bilangan prima dengan {num_digits} digit (q): {prime2}")
        totient_n = calculate_totient(prime1, prime2)
        e = generate_coprime(int(num_digits), totient_n)

        print(f"Totient n: {totient_n}")
        print(f"Bilangan prima relatif prima terhadap totient n (e): {e}")
    else  :
        print(f"Tidak ada bilangan prima dengan {num_digits} digit")


if __name__ == "__main__":
    max_digit = 3  # Initialize max_digit outside the loop
    while True:
        print("\nMenu:")
        print("1. Make keys")
        print("2. Encryption")
        print("3. Decryption")
        print("4. Custom Prime")
        print("5. Exit")

        choice = input("Enter your choice (1, 2, 3, 4, or 5): ")
        if choice == '1':
            make_key()  # Capture max_digit from make_key
        elif choice == '2':
            max_digit = encryption_menu()
        elif choice == '3':
            decryption_menu(max_digit)  # Pass max_digit to decryption_menu
        elif choice == '4':
            custom_prime_menu()
        elif choice == '5':
            print("Exiting program. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter 1, 2, 3, 4, or 5")



Menu:
1. Make keys
2. Encryption
3. Decryption
4. Custom Prime
5. Exit
Public key (e, n):  9811 70620049
Private key (d, n):  24345199 70620049

Menu:
1. Make keys
2. Encryption
3. Decryption
4. Custom Prime
5. Exit
Invalid choice. Please enter 1, 2, 3, 4, or 5

Menu:
1. Make keys
2. Encryption
3. Decryption
4. Custom Prime
5. Exit
Invalid choice. Please enter 1, 2, 3, 4, or 5

Menu:
1. Make keys
2. Encryption
3. Decryption
4. Custom Prime
5. Exit
Invalid choice. Please enter 1, 2, 3, 4, or 5

Menu:
1. Make keys
2. Encryption
3. Decryption
4. Custom Prime
5. Exit
Invalid choice. Please enter 1, 2, 3, 4, or 5

Menu:
1. Make keys
2. Encryption
3. Decryption
4. Custom Prime
5. Exit
Invalid choice. Please enter 1, 2, 3, 4, or 5

Menu:
1. Make keys
2. Encryption
3. Decryption
4. Custom Prime
5. Exit
Invalid choice. Please enter 1, 2, 3, 4, or 5

Menu:
1. Make keys
2. Encryption
3. Decryption
4. Custom Prime
5. Exit
Invalid choice. Please enter 1, 2, 3, 4, or 5

Menu:
1. Make keys
2. Encryp

KeyboardInterrupt: Interrupted by user

# Speech To Text

In [ ]:
!pip install SpeechRecognition pyttsx3 AudioSegment gTTS

Suara
1. Upload file .wav
2. Pilih path nanti diinputan
3. Plainteksnya kalian encrypt
4. Decrypt kembali
5. Terus rubah ke suara lagi

In [ ]:
import speech_recognition as sr
from gtts import gTTS

def voice_to_text(audio_file_path):
    recognizer = sr.Recognizer()

    with sr.AudioFile(audio_file_path) as source:
        audio_data = recognizer.record(source)
        text = recognizer.recognize_google(audio_data)

    return text

def text_to_speech(text, output_file_path):
    tts = gTTS(text)
    tts.save(output_file_path)

def main_menu():
    print("1. Transcribe Voice to Text")
    print("2. Text to Speech")
    print("3. Exit")

def main():
    while True:
        main_menu()
        choice = input("Enter your choice (1, 2, or 3): ")

        if choice == '1':
            audio_file_path = input("Enter the path of the audio file: ")
            transcribed_text = voice_to_text(audio_file_path)
            print("\nTranscribed Text:")
            print(transcribed_text)

        elif choice == '2':
            text = input("Enter the text to convert to speech: ")
            output_file_path = input("Enter the path to save the speech (including file extension): ")
            text_to_speech(text, output_file_path)
            print(f"Speech saved to: {output_file_path}")

        elif choice == '3':
            print("Exiting program. Goodbye!")
            break

        else:
            print("Invalid choice. Please enter 1, 2, or 3.")

if __name__ == "__main__":
    main()
